In [3]:
import numpy as np
from scipy.optimize import linprog

def is_integer(x, eps):
    return abs(x - round(x)) <= eps

def solve(A, b, c, lb, ub, eps):
    tasks_solved = 0
    stack = [(A, b, c, lb, ub)]
    optimal_solution = None
    fun_value = -np.inf

    while True:
        try:
            A, b, c, lb, ub = stack.pop()
        except IndexError:
            break
        
        solution = linprog(
            c=-c,
            A_ub=A,
            b_ub=b,
            bounds=list(zip(lb, ub)),
            method='simplex'
        )
        tasks_solved += 1

        if solution.success:
            solution_x = solution.x
            solution_sum = -solution.fun

            if all(list(map(lambda x: is_integer(x, eps), solution_x))) and solution_sum > fun_value:
                optimal_solution = solution_x
                fun_value = solution_sum
            else:
                for i, component in enumerate(solution_x):
                    if not is_integer(component, eps) and solution_sum > fun_value:
                        modified_lb = np.copy(lb)
                        modified_ub = np.copy(ub)
                        modified_ub[i] = int(component)
                        modified_lb[i] = int(component) + 1

                        stack.append((A, b, c, lb, modified_ub))
                        stack.append((A, b, c, modified_lb, ub))


    if optimal_solution is None:
        print('No solution')

    print('Optimal solution: ', optimal_solution)
    print('Sum: ', fun_value)
    print('Tasks solved: ', tasks_solved)


A = np.array([
    [4, 3],
    [-4, 3]
])
b = np.array([22, 2])
c = np.array([-5, 4])
lb = np.array([1, 0])
ub = np.array([4, 5])

# A = np.array([
#     [5, 4, -2],
#     [5, -2, 4]
# ])
# b = np.array([6, 6])
# c = np.array([20, 2, 2])
# lb = np.array([0, 0, 0])
# ub = np.array([4, 4, 4])

solve(A, b, c, lb, ub, 10 ** (-10))


Optimal solution:  [1. 2.]
Sum:  3.0
Tasks solved:  9


In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
class Task:
    def __init__(self, A, b, c, lb, ub):
        self.A = A
        self.b = b
        self.c = c
        self.lb = lb
        self.ub = ub


In [31]:
def interation():
    pass

def branch_and_bound(main_task: Task):
    stack = [main_task]
    best_solution = None
    max_value = -np.inf
    
    while len(stack) != 0:
        current_task = stack.pop()
        
        simplex_res = linprog(
            method='simplex',
            c = -current_task.c,
            b_ub = current_task.b,
            A_ub = current_task.A,
            bounds = list(zip(current_task.lb, current_task.ub))
        )
        if simplex_res.status == 0:
            real_solution = simplex_res.x
            current_sum = -simplex_res.fun
            
            if all(val.is_integer() for val in real_solution) and current_sum > max_value:
                best_solution = real_solution
                max_value = current_sum
            else:
                for index, item in enumerate(real_solution):
                    if not item.is_integer() and current_sum > max_value:
                        modified_lb = np.copy(current_task.lb)
                        modified_ub = np.copy(current_task.ub)
                        modified_ub[index] = int(item)
                        modified_lb[index] = int(item) + 1

                        stack.append(Task(current_task.A, current_task.b, current_task.c, current_task.lb, modified_ub))
                        stack.append(Task(current_task.A, current_task.b, current_task.c, modified_lb, current_task.ub))
                
            
        elif simplex_res.status == 2:
            print('Problem is infeasible.')
            return None
        elif simplex_res.status == 3:
            print('Problem is unbounded')
            return None
        
    return best_solution

In [32]:
A = np.array([
    [4, 3],
    [-4, 3]
])
b = np.array([22, 2])
c = np.array([-5, 4])
lb = np.array([1, 0])
ub = np.array([4, 5])

main_task = Task(A, b, c, lb, ub)

In [33]:
print('Solution:')

print(branch_and_bound(main_task))

Solution:
Problem is infeasible.
None
